<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#BASELINES" data-toc-modified-id="BASELINES-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>BASELINES</a></span></li><li><span><a href="#ORACLE" data-toc-modified-id="ORACLE-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>ORACLE</a></span></li><li><span><a href="#DOC-MODEL" data-toc-modified-id="DOC-MODEL-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>DOC MODEL</a></span></li><li><span><a href="#BERT-/-SUM" data-toc-modified-id="BERT-/-SUM-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>BERT / SUM</a></span></li><li><span><a href="#Ensemble" data-toc-modified-id="Ensemble-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Ensemble</a></span></li></ul></div>

Let's compare outputs from different methods

In [2]:
import pandas as pd
import pickle

In [76]:
score_dir = "../../BSDATA_V2/score_data" # Change me to your score_data file

# BASELINES

In [66]:
final_data = {}

scores = pickle.load(open('/Users/anastassiakornilova/BSDATA_V2/score_data/baseline_us_test_data_final.jsonl_2000.pkl', 'rb'))

for method in ['sumbasic', 'lsa', 'textrank']:
    basic = pd.DataFrame(scores).loc[method].to_dict()
    index = []
    value_data = {}
    for k, ds in basic.items():
        for v1, inner in ds.items():
            for v2, score in inner.items():
                value_data[(k, v1, v2)] = [score]


    data = pd.DataFrame(value_data)
    d2 = data.stack(0)
    data = d2.droplevel(0)
    final_data[(method, cutoff)] = data.describe().loc['mean']

finals = pd.DataFrame(final_data).T
(finals.reorder_levels([1,0]).loc['2000'] * 100)[[('rouge-1', 'f'), ('rouge-2', 'f'), ('rouge-l', 'f')]].round(2)

,rouge-1,rouge-2,rouge-l
,f,f,f
sumbasic,30.74,14.16,23.92
lsa,32.64,15.69,26.26
textrank,34.35,17.77,27.80


In [67]:
final_data = {}
scores = pickle.load(open('/Users/anastassiakornilova/BSDATA_V2/score_data/baseline_ca_test_data_final.jsonl_2000.pkl', 'rb'))

for method in ['sumbasic', 'lsa', 'textrank']:
    basic = pd.DataFrame(scores).loc[method].to_dict()
    index = []
    value_data = {}
    for k, ds in basic.items():
        for v1, inner in ds.items():
            for v2, score in inner.items():
                value_data[(k, v1, v2)] = [score]


    data = pd.DataFrame(value_data)
    d2 = data.stack(0)
    data = d2.droplevel(0)
    final_data[(method, cutoff)] = data.describe().loc['mean']


       
finals = pd.DataFrame(final_data).T

(finals.reorder_levels([1,0]).loc['2000'] * 100)[[('rouge-1', 'f'), ('rouge-2', 'f'), ('rouge-l', 'f')]].round(2)


,rouge-1,rouge-2,rouge-l
,f,f,f
sumbasic,35.47,16.18,29.98
lsa,35.06,16.34,29.93
textrank,35.81,18.10,29.97


In [70]:
final_data = {}

# ORACLE

In [21]:
from billsum.post_process import greedy_summarize, mmr_selection

import os
import pandas as pd 
import pickle 
from rouge import Rouge
rouge = Rouge()



In [48]:
prefix = '../../BSDATA_V2/'
locality='us'
# Load in the data
test_data = pd.read_json(prefix + 'clean_final/{}_test_data_final.jsonl'.format(locality), lines=True)

# CA data gets formatted weird - fix just in case
if 'bill_id' not in test_data.columns:
    test_data['bill_id'] = test_data['external_id']

test_data.set_index('bill_id', inplace=True)
test_sents = pickle.load(open(prefix + 'sent_data/{}_test_sent_scores.pkl'.format(locality), 'rb'))

final_scores = {}
for bill_id, sents in test_sents.items():

    doc = [v[1] for v in sents]

    scores = [v[2] for v in sents]
    scores = [s['rouge-2']['p'] for s in scores]

    sent_texts = [v[0] for v in sents]

    summary = test_data.loc[bill_id]['clean_summary']

    final_sum = ' '.join(mmr_selection(sent_texts, scores))
    rs = rouge.get_scores([final_sum],[summary])[0]

    final_scores[bill_id] = rs

for k, ds in final_scores.items():
    for v1, inner in ds.items():
        for v2, score in inner.items():
            value_data[(k, v1, v2)] = [score]


data = pd.DataFrame(value_data)
d2 = data.stack(0)
data = d2.droplevel(0)
data.describe()

rouge-1                                rouge-2               \
                 f            p            r            f            p   
count  4506.000000  4506.000000  4506.000000  4506.000000  4506.000000   
mean      0.451162     0.409217     0.581974     0.287442     0.258087   
std       0.151196     0.172050     0.179562     0.151056     0.159995   
min       0.053763     0.029412     0.033784     0.000000     0.000000   
25%       0.331217     0.281442     0.460136     0.162757     0.131630   
50%       0.448353     0.391304     0.615146     0.274065     0.226859   
75%       0.572146     0.528382     0.720688     0.402361     0.357567   
max       0.829971     0.922078     1.000000     0.748515     0.842912   

                        rouge-l                            
                 r            f            p            r  
count  4506.000000  4506.000000  4506.000000  4506.000000  
mean      0.391674     0.383756     0.382623     0.545277  
std       0.182031     0.166989     0.171348     0.183019  
min       0.000000     0.021670     0.021622     0.027027  
25%       0.254732     0.251822     0.252959     0.418605  
50%       0.404716     0.369770     0.359426     0.576271  
75%       0.525641     0.516198     0.500000     0.684211  
max       0.977376     0.824194     0.915584     1.000000

In [51]:
prefix = '../../BSDATA_V2/'
locality='ca'
# Load in the data
test_data = pd.read_json(prefix + 'clean_final/{}_test_data_final.jsonl'.format(locality), lines=True)

# CA data gets formatted weird - fix just in case
if 'bill_id' not in test_data.columns:
    test_data['bill_id'] = test_data['external_id']

test_data.set_index('bill_id', inplace=True)
test_sents = pickle.load(open(prefix + 'sent_data/{}_test_sent_scores.pkl'.format(locality), 'rb'))

final_scores = {}
for bill_id, sents in test_sents.items():

    doc = [v[1] for v in sents]

    scores = [v[2] for v in sents]
    scores = [s['rouge-2']['p'] for s in scores]

    sent_texts = [v[0] for v in sents]

    summary = test_data.loc[bill_id]['clean_summary']

    final_sum = ' '.join(mmr_selection(sent_texts, scores))
    rs = rouge.get_scores([final_sum],[summary])[0]

    final_scores[bill_id] = rs

for k, ds in final_scores.items():
    for v1, inner in ds.items():
        for v2, score in inner.items():
            value_data[(k, v1, v2)] = [score]


data = pd.DataFrame(value_data)
d2 = data.stack(0)
data = d2.droplevel(0)
data.describe()

rouge-1                                rouge-2               \
                 f            p            r            f            p   
count  4506.000000  4506.000000  4506.000000  4506.000000  4506.000000   
mean      0.493797     0.449995     0.629631     0.333616     0.308500   
std       0.143365     0.191147     0.124857     0.140890     0.175149   
min       0.053763     0.029412     0.185185     0.000000     0.000000   
25%       0.403465     0.302469     0.538568     0.231262     0.167933   
50%       0.509859     0.450000     0.633333     0.337808     0.287560   
75%       0.600000     0.593700     0.723881     0.439305     0.431818   
max       0.829971     1.000000     0.966667     0.759878     0.985612   

                        rouge-l                            
                 r            f            p            r  
count  4506.000000  4506.000000  4506.000000  4506.000000  
mean      0.438344     0.427797     0.427927     0.595821  
std       0.139147     0.164603     0.188586     0.126805  
min       0.000000     0.021670     0.021622     0.178899  
25%       0.336646     0.308569     0.282091     0.503186  
50%       0.433512     0.446708     0.423841     0.596154  
75%       0.533013     0.555294     0.568345     0.688412  
max       0.901235     0.824194     1.000000     0.966667

# DOC MODEL

In [78]:
scores = pd.read_pickle(score_dir + '/us_test_feature_model_res.pkl')
index = []
value_data = {}
for k, ds in scores.items():
    for v1, inner in ds.items():
        for v2, score in inner.items():
            value_data[(k, v1, v2)] = [score]


data = pd.DataFrame(value_data)
d2 = data.stack(0)
data = d2.droplevel(0)
data.describe()

rouge-1                                rouge-2               \
                 f            p            r            f            p   
count  3269.000000  3269.000000  3269.000000  3269.000000  3269.000000   
mean      0.385140     0.326103     0.541571     0.213765     0.179310   
std       0.108212     0.128941     0.133100     0.098893     0.102997   
min       0.046784     0.025806     0.144000     0.000000     0.000000   
25%       0.311111     0.226277     0.443182     0.140762     0.098940   
50%       0.389776     0.320513     0.535088     0.204301     0.162712   
75%       0.459574     0.415493     0.631206     0.278539     0.243542   
max       0.758621     0.817568     0.966667     0.600962     0.661538   

                        rouge-l                            
                 r            f            p            r  
count  3269.000000  3269.000000  3269.000000  3269.000000  
mean      0.323095     0.314836     0.300917     0.499184  
std       0.136487     0.120424     0.123591     0.132710  
min       0.000000     0.026053     0.025806     0.128000  
25%       0.223214     0.222234     0.205674     0.400000  
50%       0.305195     0.318446     0.293706     0.490909  
75%       0.402985     0.397708     0.384615     0.585106  
max       0.888889     0.744649     0.810811     0.966667

In [79]:
scores = pd.read_pickle(score_dir + '/ca_test_feature_model_res.pkl')
index = []
value_data = {}
for k, ds in scores.items():
    for v1, inner in ds.items():
        for v2, score in inner.items():
            value_data[(k, v1, v2)] = [score]


data = pd.DataFrame(value_data)
d2 = data.stack(0)
data = d2.droplevel(0)
data.describe()

rouge-1                                rouge-2               \
                 f            p            r            f            p   
count  1237.000000  1237.000000  1237.000000  1237.000000  1237.000000   
mean      0.383451     0.383131     0.414978     0.197641     0.204926   
std       0.086871     0.126518     0.097332     0.085951     0.113529   
min       0.074419     0.043243     0.190789     0.000000     0.000000   
25%       0.329502     0.299145     0.346369     0.135084     0.121457   
50%       0.381743     0.376543     0.402010     0.188976     0.192453   
75%       0.438356     0.461538     0.475000     0.252918     0.270386   
max       0.639098     1.000000     0.757576     0.541033     0.987552   

                        rouge-l                            
                 r            f            p            r  
count  1237.000000  1237.000000  1237.000000  1237.000000  
mean      0.213342     0.328002     0.346724     0.374255  
std       0.092188     0.091935     0.123359     0.094861  
min       0.000000     0.033144     0.032432     0.164474  
25%       0.148718     0.266734     0.260870     0.307692  
50%       0.200369     0.328292     0.338983     0.361582  
75%       0.265403     0.387962     0.423841     0.433498  
max       0.638710     0.598439     1.000000     0.737374

# BERT / SUM


In [ ]:
scores = pd.read_pickle(score_dir + '/us_bert_scores.pkl')
index = []
value_data = {}
for k, ds in scores.items():
    for v1, inner in ds.items():
        for v2, score in inner.items():
            value_data[(k, v1, v2)] = [score]


data = pd.DataFrame(value_data)
d2 = data.stack(0)
data = d2.droplevel(0)
data.describe()

# Ensemble

In [74]:
scores = pd.read_pickle('/Users/anastassiakornilova/BSDATA_V2/score_data/us_ensemble_scores.pkl')
value_data = {}
for k, ds in scores.items():
    for v1, inner in ds.items():
        for v2, score in inner.items():
            value_data[(k, v1, v2)] = [score]


data = pd.DataFrame(value_data)
d2 = data.stack(0)
data = d2.droplevel(0)
data.describe()

rouge-1                                rouge-2               \
                 f            p            r            f            p   
count  3269.000000  3269.000000  3269.000000  3269.000000  3269.000000   
mean      0.408045     0.347718     0.566831     0.238299     0.201133   
std       0.112874     0.135614     0.134506     0.106368     0.112183   
min       0.057803     0.031847     0.178899     0.000000     0.000000   
25%       0.331606     0.244898     0.469880     0.160991     0.114155   
50%       0.415094     0.342857     0.560284     0.232099     0.184615   
75%       0.486486     0.445205     0.658537     0.306533     0.274262   
max       0.782946     0.801370     0.966667     0.730159     0.662791   

                        rouge-l                            
                 r            f            p            r  
count  3269.000000  3269.000000  3269.000000  3269.000000  
mean      0.355569     0.337340     0.323110     0.526168  
std       0.142350     0.127314     0.131061     0.135368  
min       0.000000     0.021694     0.021622     0.174312  
25%       0.250000     0.242265     0.222222     0.428571  
50%       0.337500     0.341555     0.316901     0.516949  
75%       0.443350     0.429996     0.414966     0.618182  
max       0.901235     0.747498     0.773973     0.966667

In [75]:
scores = pd.read_pickle('/Users/anastassiakornilova/BSDATA_V2/score_data/ca_ensemble_scores.pkl')
value_data = {}
for k, ds in scores.items():
    for v1, inner in ds.items():
        for v2, score in inner.items():
            value_data[(k, v1, v2)] = [score]


data = pd.DataFrame(value_data)
d2 = data.stack(0)
data = d2.droplevel(0)
data.describe()

rouge-1                                rouge-2               \
                 f            p            r            f            p   
count  1237.000000  1237.000000  1237.000000  1237.000000  1237.000000   
mean      0.396520     0.398523     0.425103     0.214076     0.223440   
std       0.090687     0.131409     0.097673     0.092400     0.121173   
min       0.082353     0.047619     0.164474     0.000000     0.000000   
25%       0.340278     0.309353     0.351916     0.147874     0.132296   
50%       0.398773     0.393939     0.412844     0.207059     0.207469   
75%       0.458221     0.482993     0.489796     0.275986     0.297170   
max       0.657061     1.000000     0.761194     0.520000     0.984314   

                        rouge-l                            
                 r            f            p            r  
count  1237.000000  1237.000000  1237.000000  1237.000000  
mean      0.228227     0.340509     0.361673     0.384143  
std       0.098779     0.096799     0.129505     0.096584  
min       0.000000     0.048599     0.047619     0.151316  
25%       0.155807     0.280733     0.273256     0.314815  
50%       0.213033     0.341457     0.354430     0.369863  
75%       0.285714     0.402757     0.448052     0.445783  
max       0.712644     0.642457     1.000000     0.737374